<a href="https://colab.research.google.com/github/riemannruiz/MS_Modelado_Predictivo/blob/master/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Práctica de preprocesamiento de datos**
En este ejercicio tiene como objetivo explorar las transformaciones que se utilizan para preparar los datos antes crear un modelo de "Machine Learning".

Cargar las librerías necesarias para ejecutar el ejercicio.

In [ ]:
# Importar librerias que se usaran
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#pip install pandas-profiling[notebook]

Si el ejercicio está ejecutandose en la plataforma **Google Colab**, se debe de cargar los datos a la plataforma para poder acceder a los mismos. La carga de los datos se realizá con el código siguiente.

Si el ejercicio se está ejecutando de forma local se debe omitir el código siguiente.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Leer e importar los datos mediante la paquetería pandas y se almacenan en un DataFrame.

In [ ]:
file_path = 'glass.data'
data = pd.read_read(file_path,header=None)
cnames = ['ID','Indice_Refraccion','Na','Mg','Al','Si','K',
         'Ca','Ba','Fe','Tipo_Vidrio']
data.columns = cnames
data.head()

Conocer el estado actual de los datos sirve para entender el origen de los datos.

In [ ]:
#%% REPORTE DE CALIDAD DE LOS DATOS
def dqr(data):
    # Lista de variables de la base de datos
    cols = pd.DataFrame(list(data.columns.values),
                           columns=['Nombres'],
                           index=list(data.columns.values))
    #lista de tipos de datos
    dtyp = pd.DataFrame(data.dtypes,columns=['Tipo'])
    # lista de datos perdidos
    misval = pd.DataFrame(data.isnull().sum(),
                                  columns=['Valores_Perdidos'])
    # Lista de los datos presentes
    presval = pd.DataFrame(data.count(),
                                  columns=['Valores_Presentes'])
    # Lista de valores unicos
    unival = pd.DataFrame(columns=['Valores_Unicos'])
    # Lista de valores minimos
    minval = pd.DataFrame(columns=['Min'])
    # Lista de valores maximos
    maxval = pd.DataFrame(columns=['Max'])
    for col in list(data.columns.values):
        unival.loc[col] = [data[col].nunique()]
        try:
            minval.loc[col] = [data[col].min()]
            maxval.loc[col] = [data[col].max()]
        except:
            pass
    
    # Juntar todas las tablas
    return cols.join(dtyp).join(misval).join(presval).join(unival).join(minval).join(maxval)

reporte = dqr(data)
reporte

La descripción de los datos por medio de gráficos es muy común y es de lo mpas recomendable. Pero hay que tener encuenta detalles de escala que pueden ser engañosos para la intepretación de los datos.

In [ ]:
#%% Dibujar alguna una de las variables
#plt.figure(figsize=(5,4))
plt.scatter(data.ID,data.Indice_Refraccion)
plt.xlabel('ID'),plt.ylabel('Indice de Refraccion')
plt.grid()
plt.show()

In [ ]:
#%% Dibujar alguna una de las variables
#plt.figure(figsize=(5,4))
plt.scatter(data.ID,data.Na)
plt.xlabel('ID'),plt.ylabel('Sodio (Na)')
plt.grid()
plt.show()

In [ ]:
#%%  Dibujar una variable contra otra
plt.scatter(data.Indice_Refraccion,data.Na)
plt.xlabel('Indice de Refraccion'),plt.ylabel('Sodio (Na)')
plt.axis('square')
plt.grid()
plt.show()

# **Escalamiento de datos**
El escalameinto de los datos permite resaltar el comportamiento de lso datos sin tener la dependencia de la escla de medicón de las variables.

In [ ]:
#%% ESCALAMIENTO DE VARIABLES POR NORMALIZACION
data['Indice_Refraccion_escala'] = (data.Indice_Refraccion-data.Indice_Refraccion.mean())/data.Indice_Refraccion.std()
data['Na_escala'] = (data.Na-data.Na.mean())/data.Na.std()

### Escalamiento por medio de scikit-learn
#from sklearn import preprocessing
#data['Indice_Refraccion_escala'] = preprocessing.scale(data.Indice_Refraccion)

In [ ]:
#%% Visualizar la nueva variable
fig = plt.figure()
plt.subplot(1,2,1)
plt.scatter(data.ID,data.Na)
plt.xlabel('ID'),plt.ylabel('Sodio (Na)')
plt.title('Original')
plt.grid()
plt.subplot(1,2,2)
plt.scatter(data.ID,data.Na_escala)
plt.xlabel('Indice de Refraccion'),plt.ylabel('Sodio (Na) Reescalado')
plt.title('Rescalamiento')
plt.grid()
fig.tight_layout()
plt.show()

In [ ]:
#%% Visualizar las nuevas variables
fig = plt.figure()
plt.subplot(1,2,1)
plt.scatter(data.Indice_Refraccion,data.Na)
plt.xlabel('Indice de Refraccion'),plt.ylabel('Sodio (Na)')
#plt.axis('square')
plt.title('Original')
plt.grid()
plt.subplot(1,2,2)
plt.scatter(data.Indice_Refraccion_escala,data.Na_escala)
plt.xlabel('Indice de Refraccion'),plt.ylabel('Sodio (Na)')
#plt.axis('square')
plt.title('Rescalamiento')
plt.grid()
fig.tight_layout()
plt.show()

# **Asimetría de datos**
La asimetría de los datos se presetna cuando los datos no tienen la misma probabilidad de ocurrencia con respecto a su media. La asimetría derecha se presenta cuando se tienen más datos con valor debajo de la media cuando el fenómeno puede medirse en escalas grandes.

In [ ]:
#%% ASIMETRIA EN LAS VARIABLES
fig = plt.figure()
plt.hist(data.Indice_Refraccion,bins=30)
plt.xlabel('Indice_Refraccion'),plt.ylabel('Frecuencia')
plt.vlines(data.Indice_Refraccion.mean(),0,50,'r')
plt.show()

In [ ]:
#%% Criterio empirico para considerar que los datos pueden tener asímetria
ratio = data.max()/data.min()
ratio

In [ ]:
#%% Calculo de la asimetria
v = np.sum(np.power(data-data.mean(axis=0),2))/(data.shape[0]-1)
skewness = np.sum(np.power(data-data.mean(axis=0),3))/((data.shape[0]-1)*np.power(v,3/2))

# Calculo de la asimetria con pandas
skewness = data.skew()

# Calculo de la asimetria con scipy
from scipy import stats
skewness = stats.skew(data)

skewness

In [ ]:
#%% Verificacion del Skewness por medio de histogramas
fig = plt.figure()
plt.subplot(1,2,1)
plt.hist(data.Indice_Refraccion)
plt.xlabel('Indice_Refraccion'),plt.ylabel('Frecuencia')
plt.subplot(1,2,2)
plt.hist(data.Mg)
plt.xlabel('Magnesio (Mg)'),plt.ylabel('Frecuencia')
fig.tight_layout()
plt.show()

Limitar la asimetría de los datos puede tener consecuencias en el contenido de información de los datos.

In [ ]:
#%% Transformacion para limitar la asimetria (skewness)
data['Indice_Refraccion_no_skewness'] = np.log(data.Indice_Refraccion)
data['Indice_Refraccion_no_skewness'] = np.sqrt(data.Indice_Refraccion)
data['Indice_Refraccion_no_skewness'] = 1/data.Indice_Refraccion

## Transformacion BoxCox usando scipy
from scipy import stats
data['Indice_Refraccion_no_skewness'],lamb = stats.boxcox(data.Indice_Refraccion)

#data['Indice_Refraccion_no_skewness'] = stats.boxcox(data.Indice_Refraccion,lmbda=-5)

In [ ]:
#%% Verificacion del Skewness
fig = plt.figure()
plt.subplot(1,2,1)
plt.hist(data.Indice_Refraccion)
plt.xlabel('Indice_Refraccion'),plt.ylabel('Frecuencia')
plt.subplot(1,2,2)
plt.hist(data.Indice_Refraccion_no_skewness)
plt.xlabel('Indice_Refraccion_no_skewness'),plt.ylabel('Frecuencia')
fig.tight_layout()
plt.show()

**Perfilar datos**
Una opción para realizar los estudios más comúnes recomendados es utilizar una paqueteria especializada.

In [ ]:
#%% HACIENDO USO DEL PROFILING DE PANDAS
import pandas_profiling
report = data.profile_report()
report = pandas_profiling.ProfileReport(data)
#report.to_file(output_file="../Data/glass/Titanic data profiling.html")

# **Valores Atípicos**
Una gráfica de caja ("boxplot") es útil para identificar si se tienen datos atípicos.

In [ ]:
#%%  IDENTIFICACIÓN DE VALORES ATIPICOS
import seaborn as sns

# Un boxplot no se ve afectado por el escalamiento
sns.boxplot(y=data['Indice_Refraccion'])
#sns.boxplot(y=data['Indice_Refraccion_escala'])

Criterio para determinar datos atípicos.

In [ ]:
#%% Funcion para determinar los outliers
def find_boundaries(df_var,distance=1.5):
    IQR = df_var.quantile(0.75)-df_var.quantile(0.25)
    lower = df_var.quantile(0.25)-IQR*distance
    upper = df_var.quantile(0.75)+IQR*distance
    return lower,upper

lmin,lmax = find_boundaries(data['Indice_Refraccion'])
outliers = np.where(data['Indice_Refraccion'] > lmax, True,np.where(data['Indice_Refraccion'] < lmin, True, False))
outliers_df = data.loc[outliers, 'Indice_Refraccion']
outliers_df

Trasformar datos atípicos.

In [ ]:
#%% Haciendo la transformacion spatial sign para mitigar los outliers
tmp = data[['Indice_Refraccion_escala','Na_escala']]
#modulo = np.sqrt(np.sum(tmp*tmp,axis=1))
#tmp['Indice_Refraccion_escala'] = tmp['Indice_Refraccion_escala']/modulo
#tmp['Na_escala'] = tmp['Na_escala']/modulo
#plt.scatter(tmp.Indice_Refraccion_escala,tmp.Na_escala)

# Usando scikit-learn
from sklearn import preprocessing
tmp = preprocessing.normalize(np.array(tmp), norm='l2')
fig = plt.figure(figsize=(9,5))
plt.subplot(1,2,1)
plt.scatter(data['Indice_Refraccion_escala'],data['Na_escala'])
plt.xlabel('Indice_Refraccion_escala'),plt.ylabel('Na_escala')
plt.subplot(1,2,2)
plt.scatter(tmp[:,0],tmp[:,1])
plt.xlabel('Indice_Refraccion_escala'),plt.ylabel('Na_escala')
plt.tight_layout()
plt.show()

# **Datos perdidos**
El tratamiento de datos perdidos se debe de realizar en función de los objetivos propuestos o a las implicaciones de aplicar una técnica u otra.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
#%% Importar la tabla
datamovie = pd.read_excel('..\Data\movietest\Test de películas(1-85).xlsx')
#%% Seleccion de las columnas validas
csel = np.arange(7,244,3)
cnames = list(datamovie.columns.values[csel])
datan = datamovie[cnames]

Eliminación de datos nulos.

In [ ]:
#%% Eliminar todos los registros con datos nulos
datan_clean = datan.dropna()
miss_val_data = datan.isnull().mean().sort_values(ascending=True)

Criterio de la media, mediana o frecuencia

In [ ]:
#%% Imputacion por media o mediana
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value=99)
#imputer = SimpleImputer(strategy='constant', fill_value='Missing')
#imputer = SimpleImputer(strategy='mean')
#imputer = SimpleImputer(strategy='median')
#imputer = SimpleImputer(strategy='most_frequent')

imputer.fit(datan)
imputer.statistics_ # revisar los valores por los que remplazará
datan_clean = imputer.transform(datan)

Criterio basado en modelos

In [ ]:
#%% Imputacion iterativa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor

imputer = IterativeImputer(estimator = BayesianRidge(),max_iter=10, random_state=0)
#imputer = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=5),max_iter=10,random_state=0)
imputer.fit(datan)
data_clean = imputer.transform(datan)

# **Selección de variables**
El análisis de componentes principales puede ser utilizado para reducir la cantidad de variables que se utilizan para un análisis posterior.

In [ ]:
#%% APLICACION DE PCA A LOS DATOS
from sklearn.decomposition import PCA
from sklearn import preprocessing
tmp = preprocessing.scale(data.iloc[:,1:10])
pca = PCA()
#pca = PCA(n_components=3)
pca.fit(tmp)
data_pca = pca.transform(tmp)
componentes = pca.components_

Criterio de eliminación por varianza

In [ ]:
#%% CRITERIO DE LA VARIANZA PARA ELIMINACION DE VARIABLES
varianzas = pd.DataFrame(datan.var().sort_values(),columns=['Varianza'])
fig = plt.figure(figsize=(10,8))
plt.bar(np.arange(len(varianzas)),varianzas.Varianza)
plt.ylabel('Varianza')
plt.xticks(np.arange(len(varianzas)),varianzas.index,rotation=90)
plt.tight_layout()
plt.show()

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=1)
sel.fit_transform(X)

Criterio de eliminación por correlación

In [ ]:
#%% Analisis de correlacion para eliminacion de variables
import matplotlib.pyplot as plt
subdata = data.iloc[:,1:10]
correlaciones = np.corrcoef(subdata,rowvar=False)
plt.imshow(correlaciones)
plt.xticks(np.arange(9),np.arange(9))
plt.yticks(np.arange(9),np.arange(9))
plt.colorbar()
plt.show()

Criterio de selección por clustering jerárquico

In [ ]:
#%% Aplicación del clustering jerarquico
from scipy.cluster.hierarchy import dendrogram, linkage
Z = linkage(subdata.T,metric='correlation',method='complete')
d = dendrogram(Z)
plt.show()
correlaciones_clust = np.corrcoef(subdata.iloc[:,d['leaves']],rowvar=False)
plt.imshow(correlaciones_clust)
plt.xticks(np.arange(9),d['leaves'])
plt.yticks(np.arange(9),d['leaves'])
plt.colorbar()
plt.show()